# PyTorch dataset template

This notebook is meant to act as a template to create a custom dataset based on a downstream application (DS) index.

It requires an DS index file to be combined with a HelioFM index.  It also shows how to create a child database class based on HelioFM's database class so that all the code related to the input data is handled transparently, while the new code focuses exclusively in adding the DS information

This template uses a flare forecasting dataset as an example, casting the problem as an X-ray flux regression problem

In [ ]:
import numpy as np
import pandas as pd
import sys
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import yaml

from einops import rearrange

# Append base path.  May need to be modified if the folder structure changes.
# It gives the notebook access to the wokshop_infrastructure folder.
sys.path.append("../../")
 
# Append Surya path. May need to be modified if the folder structure changes.
# It gives the notebook access to surya's release code.
sys.path.append("../../Surya")
# from train_spectformer import get_config
from surya.utils.data import build_scalers

### Define DS dataset

This child class takes as input all expected HelioFM parameters, plus additonal parameters relevant to the downstream application.  Here we focus in particular to the DS index and parameters necessary to combine it with the HelioFM index.

Another important component of creating a dataset class for your DS is normalization.  Here we use a log normalization on xray flux that will act as the output target.  Making log10(xray_flux) strictly positive and having 66% of its values between 0 and 1

Since we are going to use this dataset moving forward, it is better to develop it as script and not as a notebook.

In [ ]:
from downstream_apps.template.datasets.template_dataset import FlareDSDataset

### Download scalers
Surya input data needs to be scaled properly for the model to work and this cell downloads the scaling information.

- If the cell below fails, try running the provided shell script directly in the terminal.
- Sometimes the download may fail due to network or server issues—if that happens, simply re-run the script a few times until it completes successfully.

In [ ]:
!sh download_scalers.sh

### Load configuration

Surya was designed to read a configuration file that defines many aspects of the model
including the data it uses we use this config file to set default values that do not
need to be modified, but also to define values specific to our downstream application

In [ ]:
# Configuration paths - modify these if your files are in different locations
config_path = "./configs/config.yaml"

# Load configuration
print("📋 Loading configuration...")
try:
    config = yaml.safe_load(open(config_path, "r"))
    config["data"]["scalers"] = yaml.safe_load(open(config["data"]["scalers_path"], "r"))
    print("✅ Configuration loaded successfully!")
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    print("Make sure config.yaml exists in your current directory")
    raise

scalers = build_scalers(info=config["data"]["scalers"])

### Initialize class

All the parameters that define a HelioFM dataset are contained within the test config file.  Scalers used to normalize HelioFm's input data are also necessary

In [ ]:
train_dataset = FlareDSDataset(
    #### All these lines are required by the parent HelioNetCDFDataset class
    index_path=config["data"]["train_data_path"],
    time_delta_input_minutes=config["data"]["time_delta_input_minutes"],
    time_delta_target_minutes=config["data"]["time_delta_target_minutes"],
    n_input_timestamps=config["model"]["time_embedding"]["time_dim"],
    rollout_steps=config["rollout_steps"],
    channels=config["data"]["channels"],
    drop_hmi_probability=config["drop_hmi_probablity"],
    use_latitude_in_learned_flow=config["use_latitude_in_learned_flow"],
    scalers=scalers,
    phase="train",
    #### Put your donwnstream (DS) specific parameters below this line
    ds_flare_index_path="./data/hek_flare_catalog.csv",
    ds_time_column="start_time",
    ds_time_tolerance = "4d",
    ds_match_direction = "forward"    
)

### Test length and return

Now we can test that the database is properly initialized and returns what is expected.  Using the toy dataset with a single day it should have a length of 1 and return a dictionary including the DS label "target"

In [ ]:
train_dataset.__len__()

In [ ]:
item = train_dataset.__getitem__(0)
item.keys()

### Define dataloader

With a working dataset we can define a dataloader.  A dataloader is simply a wrapper around a dataset that includes a sampling strategy to turn your dataset into batches.   Once we request a batch, the dataloader will return a dictionary like the dataset, but data inside will have a batch dimension

In [ ]:
data_loader = DataLoader(
                dataset=train_dataset,
                batch_size=1
            )

In [ ]:
batch = next(iter(data_loader))
batch.keys()

In [ ]:
batch['ts'].shape

### Define simple baseline model

Defining a simple baseline is important to understand what value is bringing the AI model to the problem.  In this example we define a simple logistic regression acting on the intensity of each channel.  Note that we invert the normalization to deal with strictly positive quantities

In [ ]:
class SimpleFlareModel(nn.Module):
    def __init__(self, input_dim, channel_order, scalers):
        super().__init__()
        self.linear = nn.Linear(input_dim, 1)
        self.channel_order = channel_order
        self.scalers = scalers
    
    def forward(self, x):
        # Get dimensions
        b, c, t, w, h = x.shape

        # Invert normalization
        for channel_index, channel in enumerate(self.channel_order):
            x[:, channel_index, ...] = self.scalers[channel].inverse_transform(
                x[:, channel_index, ...]
            )

        # Collapse input stack spatially
        x = x.abs().mean(dim=[3,4])

        # Rearange in preparation for linear layer
        x = rearrange(x, "b c t -> b (c t)")

        out = torch.sigmoid(self.linear(x))
        return out        

In [ ]:
b, c, t, w, h = batch['ts'].shape
model = SimpleFlareModel(c*t, config.data.channels, scalers)

In [ ]:
model.forward(batch['ts'])